In [8]:
import pandas as pd
import glob # used to read file directories, there are alternatives: pathlib.Path.rglob or os.walk

In [9]:
def load_xls_from_dir(path):
    files= glob.glob(path, recursive=True)
    df= pd.DataFrame()
    for f in files:
        #Option 1: read just the first sheet of book
        #data=pd.read_excel(f) 
        #df=df.append(data, sort=False)
        #Option 2: loop through each tab in each workbook and add id fields
        xl = pd.ExcelFile(f)
        for s in xl.sheet_names:
            data=xl.parse(s)
            # We change to lowercase as field names are case sensitive
            data.columns=[x.lower() for x in data.columns.to_list()]
            data["ori_file"]= f
            data["ori_sheet"]=s
            df=df.append(data, sort=False)
    return df
   

Often you need to consolidate several spreadsheets/tabs and you need to make sure that they have the exact same structure. This routine below does that, allows you to check fields and columns and do an initial tally of how many records the routine would be seeing before attempting to append them.

Note that if the structure is complex as in starting from the x row or column and there are artefacts like totals or values outside the main rainge, it will likely fail or bring unwanted fields/values.
There are ways of readling specific portions of a sheet, but your best bet is to try to get the format right before ingesting into a python routine.

In [10]:
def check_xls_tables_from_dir(path):
    files= glob.glob(path, recursive=True)
    df= pd.DataFrame()
    d = []
    
    for f in files:
        print(f)
        xl = pd.ExcelFile(f)
        for s in xl.sheet_names:
            data=xl.parse(s)
            data.columns=[x.lower() for x in data.columns.to_list()]
            r,c= data.shape
            ditem={
                    'ori_file': f,
                    'ori_sheet': s,
                    'ori_records':r,
                    'ori_columns':c                
                }
            for col in data.columns:
                ditem[col]=data[col].count()
            d.append( ditem)
    df=pd.DataFrame(d)
    return df
   
dfstructure= check_xls_tables_from_dir("./test_data/batch/*.xlsx")
dfstructure.head() 
    
    
    

./test_data/batch/Q2.xlsx
./test_data/batch/Q1.xlsx


,ori_file,ori_sheet,ori_records,ori_columns,customer_id,date,transaction_id,amount,result
0,./test_data/batch/Q2.xlsx,April,1967,5,1967,1967,1967,1967,1967
1,./test_data/batch/Q2.xlsx,May,2278,5,2278,2278,2278,2277,2278
2,./test_data/batch/Q2.xlsx,June,2149,5,2149,2149,2149,2149,2149
3,./test_data/batch/Q1.xlsx,Jan,1951,5,1951,1951,1951,1950,1951
4,./test_data/batch/Q1.xlsx,Feb,1881,5,1881,1881,1881,1881,1881


In [11]:
df= load_xls_from_dir("./test_data/batch/*.xlsx")
df.head()

,customer_id,date,transaction_id,amount,result,ori_file,ori_sheet
0,379017,2020-04-01,5766,5233.0,Authorised,./test_data/batch/Q2.xlsx,April
1,63289,2020-04-01,5767,13593.0,Authorised,./test_data/batch/Q2.xlsx,April
2,481336,2020-04-01,5768,18077.0,Authorised,./test_data/batch/Q2.xlsx,April
3,448090,2020-04-01,5769,18128.0,Authorised,./test_data/batch/Q2.xlsx,April
4,20708,2020-04-01,5770,38067.0,Authorised,./test_data/batch/Q2.xlsx,April


In [12]:
df.columns=[x.lower() for x in df.columns.to_list()]
df.columns=df.columns.str.replace(' ','_')
df.head()

,customer_id,date,transaction_id,amount,result,ori_file,ori_sheet
0,379017,2020-04-01,5766,5233.0,Authorised,./test_data/batch/Q2.xlsx,April
1,63289,2020-04-01,5767,13593.0,Authorised,./test_data/batch/Q2.xlsx,April
2,481336,2020-04-01,5768,18077.0,Authorised,./test_data/batch/Q2.xlsx,April
3,448090,2020-04-01,5769,18128.0,Authorised,./test_data/batch/Q2.xlsx,April
4,20708,2020-04-01,5770,38067.0,Authorised,./test_data/batch/Q2.xlsx,April


In [13]:
df.sort_values('amount', axis=0, ascending=True)

,customer_id,date,transaction_id,amount,result,ori_file,ori_sheet
31,94819,2020-02-01,1983,-5000.0,Authorised,./test_data/batch/Q1.xlsx,Feb
1437,441895,2020-02-15,3389,1.0,Authorised,./test_data/batch/Q1.xlsx,Feb
910,49009,2020-02-29,4743,1.0,Authorised,./test_data/batch/Q1.xlsx,Mar
1496,76082,2020-05-31,11505,11.0,Authorised,./test_data/batch/Q2.xlsx,June
1579,496898,2020-05-31,11588,12.0,Authorised,./test_data/batch/Q2.xlsx,June
...,...,...,...,...,...,...,...
26,55659,2020-04-02,5792,39990.0,Authorised,./test_data/batch/Q2.xlsx,April
456,230909,2020-02-05,2408,39992.0,Authorised,./test_data/batch/Q1.xlsx,Feb
1851,304071,2020-02-19,3803,39999.0,Authorised,./test_data/batch/Q1.xlsx,Feb
2277,473903,2020-05-27,10008,NaN,Authorised,./test_data/batch/Q2.xlsx,May


In [14]:
df.sort_values('date', axis=0, ascending=False, inplace=True)
df[0:10]

,customer_id,date,transaction_id,amount,result,ori_file,ori_sheet
1834,348766,2020-05-31,11843,33300.0,Authorised,./test_data/batch/Q2.xlsx,June
869,162289,2020-05-31,10878,18325.0,Authorised,./test_data/batch/Q2.xlsx,June
857,97094,2020-05-31,10866,5587.0,Authorised,./test_data/batch/Q2.xlsx,June
858,30394,2020-05-31,10867,6823.0,Authorised,./test_data/batch/Q2.xlsx,June
859,422636,2020-05-31,10868,23120.0,Authorised,./test_data/batch/Q2.xlsx,June
860,429733,2020-05-31,10869,17023.0,Authorised,./test_data/batch/Q2.xlsx,June
861,195961,2020-05-31,10870,12496.0,Authorised,./test_data/batch/Q2.xlsx,June
862,348365,2020-05-31,10871,34061.0,Authorised,./test_data/batch/Q2.xlsx,June
863,95928,2020-05-31,10872,22681.0,Authorised,./test_data/batch/Q2.xlsx,June
864,164313,2020-05-31,10873,4609.0,Authorised,./test_data/batch/Q2.xlsx,June


Lets look for some duplicates

In [15]:
df[df.duplicated(keep=False)] # first full record

,customer_id,date,transaction_id,amount,result,ori_file,ori_sheet
20,209024,2020-05-01,7752,15583.0,Authorised,./test_data/batch/Q2.xlsx,May
21,209024,2020-05-01,7752,15583.0,Authorised,./test_data/batch/Q2.xlsx,May


In [16]:
df[df.transaction_id.duplicated(keep=False)] # transaction id

,customer_id,date,transaction_id,amount,result,ori_file,ori_sheet
19,339698,2020-05-01,7752,10957.0,Authorised,./test_data/batch/Q2.xlsx,May
20,209024,2020-05-01,7752,15583.0,Authorised,./test_data/batch/Q2.xlsx,May
21,209024,2020-05-01,7752,15583.0,Authorised,./test_data/batch/Q2.xlsx,May


In [17]:
df2= df[df[["amount","customer_id"]].duplicated(keep=False)] # transaction id
df2

,customer_id,date,transaction_id,amount,result,ori_file,ori_sheet
20,209024,2020-05-01,7752,15583.0,Authorised,./test_data/batch/Q2.xlsx,May
21,209024,2020-05-01,7752,15583.0,Authorised,./test_data/batch/Q2.xlsx,May


In [18]:
df2.to_excel("./output/101_Output_Exceptions.xlsx", sheet_name='Duplicates')